# InvarLock: Certificate Deep Dive

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/invarlock/invarlock/blob/main/notebooks/invarlock_certificate_deep_dive.ipynb)

**Purpose:** Learn how to read `evaluation.cert.json` and understand what `PASS/FAIL` means.
**Runtime:** ~5–10 minutes

This notebook can generate a small demo certificate, then walks the major sections:

- `validation` (gates)
- `primary_metric` (ratios + CIs)
- `invariants` / `spectral` / `rmt` / `variance` guard summaries
- `provenance` (digests, seeds, policy snapshot pointers)


In [ ]:
!pip -q install "invarlock[hf]"
import sys
import invarlock

print("Python:", sys.version.split()[0])
print("InvarLock:", invarlock.__version__)


In [ ]:
!invarlock doctor --json || true


## Generate a demo certificate (small window counts)


In [ ]:
%%bash
cat > demo_preset.yaml <<'YAML'
dataset:
  provider: wikitext2
  split: validation
  seq_len: 256
  stride: 256
  preview_n: 24
  final_n: 24
  seed: 42
eval:
  metric:
    kind: ppl_causal
YAML

cat > quant_rtn_demo.yaml <<'YAML'
edit:
  name: quant_rtn
  plan:
    bitwidth: 8
    per_channel: true
    group_size: 128
    clamp_ratio: 0.005
    scope: attn
YAML

export INVARLOCK_ALLOW_NETWORK=1 INVARLOCK_DEDUP_TEXTS=1 INVARLOCK_TINY_RELAX=1 \
  TRANSFORMERS_NO_TORCHVISION=1 TOKENIZERS_PARALLELISM=false
invarlock certify \
  --baseline gpt2 \
  --subject  gpt2 \
  --adapter  auto \
  --profile  dev \
  --tier     balanced \
  --preset   demo_preset.yaml \
  --out      runs/cert_deep_dive \
  --cert-out reports/cert_deep_dive \
  --edit-config quant_rtn_demo.yaml


## Load the certificate


In [ ]:
import json
from pathlib import Path

CERT_PATH = Path('reports/cert_deep_dive/evaluation.cert.json')
cert = json.loads(CERT_PATH.read_text(encoding='utf-8'))
print('Loaded:', CERT_PATH)


## Safety dashboard (PASS/FAIL)


In [ ]:
validation = cert.get('validation', {})
print('overall_pass:', validation.get('overall_pass'))
print('failed gates:', [k for k,v in validation.items() if k!='overall_pass' and v is False])


## Primary metric


In [ ]:
pm = cert.get('primary_metric', {})
for key in ('kind', 'preview', 'final', 'ratio_vs_baseline', 'ratio_ci', 'ratio_ci_level'):
    if key in pm:
        print(f'{key}:', pm.get(key))


## Guard summaries


In [ ]:
for section in ('invariants', 'spectral', 'rmt', 'variance'):
    block = cert.get(section, {})
    verdict = (block.get('passed') if isinstance(block, dict) else None)
    print(section, 'passed=' + str(verdict))


## Provenance


In [ ]:
meta = cert.get('meta', {})
prov = cert.get('provenance', {})
print('meta.model_id:', meta.get('model_id'))
print('meta.adapter:', meta.get('adapter'))
print('meta.device:', meta.get('device'))
print('meta.profile:', meta.get('profile'))
print('provider_digest:', prov.get('provider_digest'))
print('policy_digest:', prov.get('policy_digest'))


## Verification walkthrough


In [ ]:
%%bash
invarlock verify --json reports/cert_deep_dive/evaluation.cert.json


## Related docs

- `docs/user-guide/reading-certificate.md`
- `docs/reference/certificates.md`
